## Preprocessing of Data and Augmentation
- smote_"name of file" e.g. smote_C4M1 (type = numpy array) for normalized data augmentation with SMOTE (Synthetic Minority Oversampling Technique)
- gauss_data_"name of file" e.g gauss_data_C3M2 (type = numpy array) for normaliezed data augmentation with gaussian noise

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn import preprocessing
import torch

## Read in Data:

In [2]:
#list all files of patient x
dirname = '../data/Exercises_SS22/sleeplab_dataset_10hz/patient_29_male_7_years'
readings = []
names = []

for filename in os.listdir(dirname):
    f = os.path.join(dirname, filename)
    x = f.replace('\\', '/')
    readings.append(x)
    f = filename.replace('.csv','')
    names.append(f)



In [3]:
measurements = [pd.read_csv(i, skiprows=1, names=[names[ix]]) for ix, i in enumerate(readings[:-1])]
label = pd.read_csv(readings[-1], usecols=['Schlafstadium'])
data = pd.concat(measurements, axis=1)

In [4]:
converted_label = label.replace(['WK', 'REM', 'N1', 'N2', 'N3'], [0, 1, 2, 3, 4])
converted_label

,Schlafstadium
0,0
1,0
2,0
3,0
4,0
...,...
1087,0
1088,0
1089,0
1090,0


In [5]:
normalized_df=(data-data.mean())/data.std()

In [6]:
normalized_df

,BeinLi_10HZ,BeinRe_10HZ,C3M2_10HZ,C4M1_10HZ,EMG_10HZ,F3M2_10HZ,F4M1_10HZ,LEOGM2_10HZ,O1M2_10HZ,O2M1_10HZ,REOGM1_10HZ
0,0.013138,0.008852,0.106808,-0.115749,0.019222,-0.078712,0.105964,0.129148,-0.346040,0.153007,0.635409
1,0.013138,0.008852,0.106808,-0.115749,0.019222,-0.078712,0.105964,0.129148,-0.346040,0.153007,0.635409
2,0.013138,0.008852,0.106808,-0.115749,0.019222,-0.078712,0.105964,0.129148,-0.346040,0.153007,0.635409
3,0.013138,0.008852,0.106808,-0.115749,0.019222,-0.078712,0.105964,0.129148,-0.346040,0.153007,0.635409
4,0.013138,0.008852,0.106808,-0.115749,0.019222,-0.078712,0.105964,0.129148,-0.346040,0.153007,0.635409
...,...,...,...,...,...,...,...,...,...,...,...
327305,0.272937,0.379179,0.284994,0.397626,0.213423,-0.017849,0.735155,-0.475123,0.009262,0.321457,1.367751
327306,-0.246661,-0.361474,0.245397,0.333454,1.378628,0.529919,0.828369,0.250002,0.301864,0.417714,0.696438
327307,0.013138,0.502621,0.938339,1.017954,-0.174979,0.996536,1.154616,0.572280,0.260064,-0.376409,0.482838
327308,0.143037,0.132294,0.819549,0.419017,1.767030,0.935673,0.688549,0.491710,0.239164,-0.135765,0.116667


In [7]:
segments = np.array([[i] * 300 for i in range(len(converted_label))]).flatten()[:normalized_df.shape[0]]

tuples = list(zip(segments, normalized_df.index))

index = pd.MultiIndex.from_tuples(tuples, names=["Samples", "Datapoints"])
multi_index_df = normalized_df.set_index(index)

In [8]:
multi_index_df

BeinLi_10HZ  BeinRe_10HZ  C3M2_10HZ  C4M1_10HZ  EMG_10HZ  \
Samples Datapoints                                                             
0       0              0.013138     0.008852   0.106808  -0.115749  0.019222   
        1              0.013138     0.008852   0.106808  -0.115749  0.019222   
        2              0.013138     0.008852   0.106808  -0.115749  0.019222   
        3              0.013138     0.008852   0.106808  -0.115749  0.019222   
        4              0.013138     0.008852   0.106808  -0.115749  0.019222   
...                         ...          ...        ...        ...       ...   
1091    327305         0.272937     0.379179   0.284994   0.397626  0.213423   
        327306        -0.246661    -0.361474   0.245397   0.333454  1.378628   
        327307         0.013138     0.502621   0.938339   1.017954 -0.174979   
        327308         0.143037     0.132294   0.819549   0.419017  1.767030   
        327309         0.272937     0.255737   0.720557   0.739876  1.572829   

                    F3M2_10HZ  F4M1_10HZ  LEOGM2_10HZ  O1M2_10HZ  O2M1_10HZ  \
Samples Datapoints                                                            
0       0           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        1           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        2           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        3           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        4           -0.078712   0.105964     0.129148  -0.346040   0.153007   
...                       ...        ...          ...        ...        ...   
1091    327305      -0.017849   0.735155    -0.475123   0.009262   0.321457   
        327306       0.529919   0.828369     0.250002   0.301864   0.417714   
        327307       0.996536   1.154616     0.572280   0.260064  -0.376409   
        327308       0.935673   0.688549     0.491710   0.239164  -0.135765   
        327309       0.712508   0.991493     0.511853   0.280964   0.032685   

                    REOGM1_10HZ  
Samples Datapoints               
0       0              0.635409  
        1              0.635409  
        2              0.635409  
        3              0.635409  
        4              0.635409  
...                         ...  
1091    327305         1.367751  
        327306         0.696438  
        327307         0.482838  
        327308         0.116667  
        327309         0.543866  

[327310 rows x 11 columns]

In [9]:
counted = multi_index_df.groupby(level=0).count()

In [10]:
smallSampleIndices = counted.loc[counted.BeinLi_10HZ < 300].index
if len(smallSampleIndices) > 0:
    multi_index_df = multi_index_df.drop(smallSampleIndices)
    converted_label = converted_label.drop(smallSampleIndices)

In [11]:
multi_index_df

BeinLi_10HZ  BeinRe_10HZ  C3M2_10HZ  C4M1_10HZ  EMG_10HZ  \
Samples Datapoints                                                             
0       0              0.013138     0.008852   0.106808  -0.115749  0.019222   
        1              0.013138     0.008852   0.106808  -0.115749  0.019222   
        2              0.013138     0.008852   0.106808  -0.115749  0.019222   
        3              0.013138     0.008852   0.106808  -0.115749  0.019222   
        4              0.013138     0.008852   0.106808  -0.115749  0.019222   
...                         ...          ...        ...        ...       ...   
1090    327295        -0.246661     0.008852   0.284994   0.675704  0.019222   
        327296         0.013138     0.132294  -0.546537   0.419017  0.990226   
        327297         0.662634     0.255737  -0.704924   0.333454 -0.951782   
        327298        -0.506459     0.132294  -0.249562   0.825439 -0.563381   
        327299         0.143037    -0.238032  -0.328755   0.033985  0.019222   

                    F3M2_10HZ  F4M1_10HZ  LEOGM2_10HZ  O1M2_10HZ  O2M1_10HZ  \
Samples Datapoints                                                            
0       0           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        1           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        2           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        3           -0.078712   0.105964     0.129148  -0.346040   0.153007   
        4           -0.078712   0.105964     0.129148  -0.346040   0.153007   
...                       ...        ...          ...        ...        ...   
1090    327295      -0.443891   0.781762    -3.718044  -0.074338   0.297393   
        327296      -1.052522   0.269088    -3.577047  -0.157938   0.706486   
        327297      -1.356837   0.432211    -3.234627  -0.680441   0.104878   
        327298      -0.484466   0.921583    -2.207366  -0.304239   0.369586   
        327299      -0.261301   0.292391    -1.723950  -0.325139  -0.183894   

                    REOGM1_10HZ  
Samples Datapoints               
0       0              0.635409  
        1              0.635409  
        2              0.635409  
        3              0.635409  
        4              0.635409  
...                         ...  
1090    327295         3.930950  
        327296         2.588322  
        327297         2.679865  
        327298         2.374722  
        327299         1.978037  

[327300 rows x 11 columns]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_ix, test_ix = train_test_split(multi_index_df.index.levels[0][:-1], random_state=42)

In [14]:
train_X = multi_index_df.loc[train_ix]
train_y = converted_label.loc[train_ix]
test_X = multi_index_df.loc[test_ix]
test_y = converted_label.loc[test_ix]

In [15]:
train_X.reset_index(0)[train_X.reset_index(0).Samples==92].values
list(set(train_X.reset_index(0).Samples))[10]

15

In [16]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, label_df):
        self.dataframe = dataframe
        self.label_df = label_df

    def __len__(self):
        return len(self.label_df)

    def __getitem__(self, idx):
        output = torch.tensor(self.dataframe.loc[list(set(self.dataframe.reset_index(0).Samples))[idx]].values.astype(np.float32)).unsqueeze(0)
        label = self.label_df.iloc[idx].values[0]
        return output, label

In [18]:
train_dataset = CustomDataset(train_X, train_y)
test_dataset = CustomDataset(test_X, test_y)

In [24]:
import torch.nn as nn
import torch.nn.functional as F

class ClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [25]:
class SleepClassification(ClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Softmax()
        )
    
    def forward(self, xb):
        return self.network(xb)

In [26]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

model = SleepClassification()

num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dataloader, test_dataloader, opt_func)

torch.Size([1, 1, 300, 11])


C:\Users\nils0\anaconda3\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

...

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 30

torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
torch.Size([1, 1, 300, 11])
